In [ ]:
using PowerSystems
using PowerSimulations
using StorageSystemsSimulations
using HiGHS
# using Gurobi
using Dates
using DataFrames

WebIO._IJuliaInit()

## System Diagrams


**Copper Plate**

```mermaid
---

config:
  look: handDrawn
  theme: neutral
  flowchart:
      curve: linear
---
graph LR;
    wind-1((wind-1)) --- bus-1;
    storage-1((storage-1)) --- bus-1;
    bus-1 --- load-1((load-1));
    bus-1 --- gas-1((gas-1));
```

---

**Storage near Wind**

```mermaid
---
config:
  look: handDrawn
  theme: neutral
  flowchart:
      curve: linear
---
graph LR;
    bus-1[bus-1] ---- tr-1[[transmission constraint]]
    tr-1 ---- bus-2[bus-2];
    wind-1((wind-1)) --- bus-1;
    storage-1((storage-1)) --- bus-1;
    bus-2 --- load-1((load-1));
    bus-2 --- gas-1((gas-1));
```

---

**Storage near Load**

```mermaid
---
config:
  look: handDrawn
  theme: neutral
  flowchart:
      curve: linear
---
graph LR;
    bus-1[bus-1] ---- tr-1[[transmission constraint]]
    tr-1 ---- bus-2[bus-2];
    wind-1((wind-1)) --- bus-1;
    bus-2 --- storage-1((storage-1));
    bus-2 --- load-1((load-1));
    bus-2 --- gas-1((gas-1));
```

## Build System

### Components

In [ ]:
## This function builds the system, it has an optional argument that if set 
## to true will include a storage resource in the system
function make_system(;storage::Bool=false) 
    ## areas define abstract regions where reources can be located
    area1 = Area(
        name = "area-1",
        peak_active_power = 30, # There is no load in area-1 so this is ignored
    )
    area2 = Area(
        name = "area-2",
        peak_active_power = 30, # The time series data in the file load/load-1-rt.csv for load in area-2 
                                # is scaled to be between 0 and 1. When it is read by SIENNA it scales
                                # the peak load to be this value.
    )
    bus1 = ACBus(
        number = 1,
        name = "bus-1",
        bustype = ACBusTypes.REF,
        angle = 0.0, # Required by not used for static analysis
        magnitude = 1.0, # Required by not used for static analysis
        voltage_limits = (min = 0.9, max = 1.05), # Required by not used for static analysis
        base_voltage = 230.0, # Required by not used for static analysis
        area = area1
    )
    bus2 = ACBus(
        number = 2,
        name = "bus-2",
        bustype = ACBusTypes.REF,
        angle = 0.0, # Required by not used for static analysis
        magnitude = 1.0, # Required by not used for static analysis
        voltage_limits = (min = 0.9, max = 1.05), # Required by not used for static analysis
        base_voltage = 230.0, # Required by not used for static analysis
        area = area2
    )
    line1 = Line(
        name = "line-1",
        available = true,
        active_power_flow = 0.0, # Inital value for model
        reactive_power_flow = 0.0, # Required by not used for static analysis
        arc = Arc(; from = bus1, to = bus2),
        r = 0.00281, # Required by not used for static analysis
        x = 0.0281, # Required by not used for static analysis
        b = (from = 0.00356, to = 0.00356), # Required by not used for static analysis
        rating = 0.3,
        angle_limits = (min = -0.7, max = 0.7), # Required by not used for static analysis
    )
    
    # Generation
    wind1 = RenewableDispatch(
        name = "wind-1",
        available = true,
        bus = bus1,
        active_power = 0.0, # Inital value for model
        reactive_power = 0.0, # Required by not used for static analysis
        rating = 40.0, # Nameplate capacity
        prime_mover_type = PrimeMovers.WT,
        reactive_power_limits = nothing, # Required by not used for static analysis
        power_factor = 1.0, # Required by not used for static analysis
        operation_cost = RenewableGenerationCost(; variable = CostCurve(; value_curve = LinearCurve(1.0))),
        base_power = 1.0 # Setting base power to 1 allows you to imput other values in natural units
    )

    storage1 = EnergyReservoirStorage(
        name = "storage-1",
        available = true,
        bus = bus1,
        prime_mover_type = PrimeMovers.BA,
        storage_technology_type = StorageTech.LIB,
        storage_capacity = 4.0, #MWh 
        storage_level_limits = (0.0, 1.0),
        initial_storage_capacity_level = 0.0,
        rating = 1.0,
        active_power = 0.0, # Inital value for model
        reactive_power = 0.0, # Required by not used for static analysis
        reactive_power_limits = nothing,
        input_active_power_limits = (0.0, 2.0),
        output_active_power_limits = (0.0, 2.0),
        efficiency = (in=1.0, out=1.0),
        base_power = 1.0, # Setting base power to 1 allows you to imput other values in natural units
        operation_cost = StorageCost(; energy_shortage_cost = 50.0, energy_surplus_cost = 40.0),
    )
    
    gas1 = ThermalStandard(
        name = "gas-1",
        available = true,
        status = true,
        bus = bus2,
        active_power = 0.0, # Inital value for model
        reactive_power = 0.0, # Required by not used for static analysis
        rating = 30.0,
        active_power_limits = (min = 0.0, max = 30.0),
        reactive_power_limits = nothing, # Required by not used for static analysis
        ramp_limits = nothing, # Can ramp up or down at any rate
        operation_cost = ThermalGenerationCost(variable = CostCurve(; value_curve = LinearCurve(10.0)), fixed = 0.0, start_up = 0.0, shut_down = 0.0),
        base_power = 1.0, # Setting base power to 1 allows you to imput other values in natural units
        time_limits = (up = 0.0, down = 0.0), # No limits on how long unit can be on or off
        prime_mover_type = PrimeMovers.CC,
        fuel = ThermalFuels.NATURAL_GAS,
    )
    
    #Load
    load1 = PowerLoad(
        name = "load-1",
        available = true,
        bus = bus2,
        active_power = 0.0, # Inital value for model
        reactive_power = 0.0, # Required by not used for static analysis
        base_power = 1.0, # Setting base power to 1 allows you to imput other values in natural units
        max_active_power = 30.0,
        max_reactive_power = 0.0, # Required by not used for static analysis
    )
    
    system = System(100.0);
    # clear_components!(system)
    add_components!(system, [area1, area2, bus1, bus2, line1])
    add_components!(system, [wind1, gas1])
    if storage 
        add_components!(system, [storage1])
    end
    add_components!(system, [load1])

    clear_time_series!(system)
    add_time_series!(system, "./timeseries-pointers-rt.json")
    transform_single_time_series!(system, Dates.Day(7), Dates.Day(7))

    return(system)
end

make_system (generic function with 1 method)

In [3]:
system = make_system(storage=true)

[ Info: Cleared all time series.
┌ Warning: There is only one forecast window. Setting interval = empty period
└ @ InfrastructureSystems C:\Users\UEU112259C\.julia\packages\InfrastructureSystems\LEg3t\src\system_data.jl:696
┌ Warning: There is only one forecast window. Setting interval = empty period
└ @ InfrastructureSystems C:\Users\UEU112259C\.julia\packages\InfrastructureSystems\LEg3t\src\system_data.jl:696


Property,Value
Name,
Description,
System Units Base,SYSTEM_BASE
Base Power,100.0
Base Frequency,60.0
Num Components,10
Type,Count
ACBus,2
Arc,1
Area,2


## Problem Formation and Solution

### Define Problem

In [ ]:
template = template_economic_dispatch(); # Do economic dispatch

set_network_model!(
    template,
    NetworkModel(CopperPlatePowerModel, use_slacks = true) # CopperPlate mean "ignore transmission constraints"
)
storage_model = DeviceModel(
    EnergyReservoirStorage,
    StorageDispatchWithReserves;
    attributes=Dict(
        "reservation" => false,
        "energy_target" => false,
        "cycling_limits" => false,
        "regularization" => false,
    ),
)
set_device_model!(template, storage_model)
template

Network Model
┌───────────────┬───────────────────────┐
│ Network Model │ CopperPlatePowerModel │
│ Slacks        │ true                  │
│ PTDF          │ false                 │
│ Duals         │ None                  │
└───────────────┴───────────────────────┘

Device Models
┌────────────────────────┬─────────────────────────────┬────────┐
│ Device Type            │ Formulation                 │ Slacks │
├────────────────────────┼─────────────────────────────┼────────┤
│ RenewableNonDispatch   │ FixedOutput                 │ false  │
│ ThermalStandard        │ ThermalBasicDispatch        │ false  │
│ PowerLoad              │ StaticPowerLoad             │ false  │
│ InterruptiblePowerLoad │ PowerLoadInterruption       │ false  │
│ RenewableDispatch      │ RenewableFullDispatch       │ false  │
│ EnergyReservoirStorage │ StorageDispatchWithReserves │ false  │
└────────────────────────┴─────────────────────────────┴────────┘

Branch Models
┌─────────────────────┬─────────────────────────┬────────┐
│ Branch Type         │ Formulation             │ Slacks │
├─────────────────────┼─────────────────────────┼────────┤
│ Line                │ StaticBranch            │ false  │
│ TapTransformer      │ StaticBranch            │ false  │
│ Transformer2W       │ StaticBranch            │ false  │
│ TwoTerminalHVDCLine │ HVDCTwoTerminalDispatch │ false  │
└─────────────────────┴─────────────────────────┴────────┘

Service Models
┌──────────────────────────────┬──────────────┬────────┬──────────────────┐
│ Service Type                 │ Formulation  │ Slacks │ Aggregated Model │
├──────────────────────────────┼──────────────┼────────┼──────────────────┤
│ VariableReserve{ReserveUp}   │ RangeReserve │ false  │ true             │
│ VariableReserve{ReserveDown} │ RangeReserve │ false  │ true             │
└──────────────────────────────┴──────────────┴────────┴──────────────────┘

In [5]:
# solver = optimizer_with_attributes(Gurobi.Optimizer, "MIPGap" => 0.5) 
solver = optimizer_with_attributes(HiGHS.Optimizer, "mip_rel_gap" => 0.5)
problem = DecisionModel(
    template,
    system;
    optimizer = solver,
    horizon = Day(7),
    optimizer_solve_log_print = true,
    calculate_conflict = true,
    store_variable_names = true,
)
build!(problem; output_dir = mktempdir())

InfrastructureSystems.Optimization.ModelBuildStatusModule.ModelBuildStatus.BUILT = 0

### Solve Problem

In [6]:
solve!(problem)

Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 1848 rows; 1176 cols; 3191 nonzeros
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [1e+02, 1e+06]
  Bound  [2e-02, 4e-01]
  RHS    [2e-02, 4e-01]
Solving LP without presolve, or with basis, or unconstrained
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Ph1: 0(0) 0s
        559     9.5945444455e+03 Pr: 0(0) 0s
Model status        : Optimal
Simplex   iterations: 559
Objective value     :  9.5945444455e+03
P-D objective error :  6.6351572924e-16
HiGHS run time      :          0.01


InfrastructureSystems.Simulation.RunStatusModule.RunStatus.SUCCESSFULLY_FINALIZED = 0

### Results

In [7]:
res = OptimizationProblemResults(problem)

StorageEnergyOutput__EnergyReservoirStorage
ActivePowerBalance__System
ProductionCostExpression__ThermalStandard
ProductionCostExpression__RenewableDispatch
ActivePowerTimeSeriesParameter__PowerLoad
ActivePowerTimeSeriesParameter__RenewableDispatch
ActivePowerOutVariable__EnergyReservoirStorage
SystemBalanceSlackUp__System
ActivePowerInVariable__EnergyReservoirStorage
ActivePowerVariable__RenewableDispatch
SystemBalanceSlackDown__System


In [10]:
series = leftjoin(leftjoin(leftjoin(
    read_parameter(res, "ActivePowerTimeSeriesParameter__PowerLoad"),
    read_variable(res, "ActivePowerVariable__RenewableDispatch"),
    on = :DateTime
    ),
    read_variable(res, "ActivePowerVariable__ThermalStandard"),
    on = :DateTime
    ),
    read_variable(res, "EnergyVariable__EnergyReservoirStorage"),
    on = :DateTime
)

println(series)
# Load (MW), Wind Generation (MW), Gas Gneration (MW), Storage stored energy (kWh)

168×5 DataFrame
 Row │ DateTime             load-1    wind-1    gas-1     storage-1 
     │ DateTime             Float64   Float64?  Float64?  Float64?  
─────┼──────────────────────────────────────────────────────────────
   1 │ 2020-01-01T00:00:00  -22.0531   23.2336   0.0        1.18051
   2 │ 2020-01-01T01:00:00  -22.0729   24.0729   0.0        3.18051
   3 │ 2020-01-01T02:00:00  -22.4083   20.4083   0.0        1.18051
   4 │ 2020-01-01T03:00:00  -23.2921   25.2921   0.0        3.18051
   5 │ 2020-01-01T04:00:00  -25.5965   26.416    0.0        4.0
   6 │ 2020-01-01T05:00:00  -28.686    22.5482   6.13783    4.0
   7 │ 2020-01-01T06:00:00  -30.0      20.4327   7.56731    2.0
   8 │ 2020-01-01T07:00:00  -29.1477   20.4299   6.71775   -0.0
   9 │ 2020-01-01T08:00:00  -27.5891   21.5447   6.04436    0.0
  10 │ 2020-01-01T09:00:00  -26.5632   18.7074   9.8558     2.0
  11 │ 2020-01-01T10:00:00  -25.8016   13.7583  10.0433     0.0
  12 │ 2020-01-01T11:00:00  -25.2177   11.2881  13.9295  